In [4]:
from PIL import Image
import sys
import numpy as np
from src.helper import readImage, RGBtoGRAY, makeShape8Divisible, changeRange, removePadding, sharpenImage,saveImage
import matplotlib.pyplot as plt
from scipy.fftpack import dct, dst
from src.seam import removeSeam, insertSeam
from src.transforms import DCT, DST 
import os
from pathlib import Path
import matplotlib.pyplot as plt


In [2]:
def dynamicCost(ImportanceMap,h,w):
    costArray=np.zeros((h,w))
    costArray[-1,:]=np.array(ImportanceMap[-1])
    paths=[[] for i in range(w)]
    
    for i in range(h-2,-2,-1):
        # for first column
        m=0 if(costArray[i+1,0]<costArray[i+1,1]) else 1
        if(i<0):
            paths[0]=paths[m][:h-2-i].copy()
            paths[0].append((i+1,m))
        else:
            costArray[i,0] = ImportanceMap[i][0] + costArray[i+1,m]
            paths[0]=paths[m][:h-2-i].copy()
            paths[0].append((i+1,m))
        
        # for last column
        m=-1 if(costArray[i+1,-1]<costArray[i+1,-2]) else -2
        if(i<0):
            paths[-1].append((i+1,m))
        else:
            costArray[i,-1] = ImportanceMap[i][-1] + costArray[i+1,m]
            paths[-1].append((i+1,m))

        #for rest of the columns
        for j in range(1,w-1,1):
            m=j-1 if(costArray[i+1,j-1]<costArray[i+1,j] and costArray[i+1,j-1]<costArray[i+1,j+1]) else j if(costArray[i+1,j]<costArray[i+1,j+1]) else j+1
            if(i<0):
                paths[j]=paths[m][:h-2-i].copy()
                paths[j].append((i+1,m))
            else:
                costArray[i,j] = ImportanceMap[i][j] + costArray[i+1,m]
                paths[j]=paths[m][:h-2-i].copy()
                paths[j].append((i+1,m))
    
    return (costArray,paths)

In [5]:
from tqdm import tqdm

importanceMapPath='/home/scai/msr/aiy227514/scratch/Assignment783/Part2/Dataset/ImportanceMaps'
targetPath='/home/scai/msr/aiy227514/scratch/Assignment783/Part2/Dataset/CumulativeMinimumEnergy'
names = os.listdir(importanceMapPath)
for i in tqdm(names):
    img=np.array(Image.open(os.path.join(importanceMapPath,i)))
    costImage,_= dynamicCost(img, 512,512)
    costImage = 255 * (costImage - costImage.min()) / (costImage.max() - costImage.min())
    # Convert to uint8
    costImage = costImage.astype(np.uint8)
    tempIM = Image.fromarray(costImage)
    tempIM.save(os.path.join(targetPath,i))
#     plt.imshow(costImage)
#     plt.show()
#     break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 296/296 [04:09<00:00,  1.19it/s]


In [1]:
from transforms2 import DCT, DST
from helper import RGBtoGRAY


def getImportanceMap(img_RGB):
    cosineTransform=DCT()
    sineTransform=DST()

    # to perform DCT we need to first get the grayscale of the image
    img_gray=RGBtoGRAY(img_RGB)

    # #Performing DCT
    DCT_transformed_img= cosineTransform.transform(cosineTransform.transform(img_gray).T ).T

    partialX= sineTransform.transform( cosineTransform.transform(img_gray=DCT_transformed_img) )
    partialY =sineTransform.transform( cosineTransform.transform(img_gray=DCT_transformed_img.T) ).T
    #calculating Importance map based on partial derivatives
    ImportanceMap=np.abs(partialX)+np.abs(partialY)
    ImportanceMap = 255 * (ImportanceMap - ImportanceMap.min()) / (ImportanceMap.max() - ImportanceMap.min())
    
    return ImportanceMap

In [8]:
from tqdm import tqdm
import os
import numpy as np
GT='/home/scai/msr/aiy227514/scratch/Assignment783/Part2/Dataset/PERCEPT'
targetPath='/home/scai/msr/aiy227514/scratch/Assignment783/Part2/Dataset/ImportanceMapAfterSC'
names = os.listdir(GT)
for i in tqdm(names):
    img=np.array(Image.open(os.path.join(GT,i)))
    tempIM = Image.fromarray(getImportanceMap(img).astype(np.uint8))
    tempIM.save(os.path.join(targetPath,i))

100%|█████████████████████████████████████████| 296/296 [05:26<00:00,  1.10s/it]
